# 切分模型

在实际生产环境中有很多需求需要将模型切分成多个部分，比如[ESSM](https://arxiv.org/pdf/1804.07931.pdf)模型中有两个模型联合训练，在这种情况下为了满足在线预估的需求，需要将这两个模型单独保存，在线逻辑会在不同的server中加载并inference。

将模型切分后也非常有助于简化在线推理的工作，下面我们通过wide_deep模型来看在TensorFlow中如何切分模型，并与TensorNet结合使用。这部分代码放在[examples/wide_deep.py](../../examples/wide_deep.py)，可以直接执行测试。

下面这部分代码与[quick start with wide deep](01-begin-with-wide-deep.ipynb)展示的一致。

In [6]:
import sys
sys.path.append('/da2/zhangyansheng/tensornet') # 在此设置您的tensornet包的位置

In [13]:
import os
from datetime import datetime

import tensorflow as tf
import tensornet as tn
import numpy as np


class Config(object):
    DATA_DIR = './data/'
    FILE_MATCH_PATTERN = "tf-*"

    MODEL_DIR = "./model/"

    BATCH_SIZE = 32

    TRAIN_DAYS = ['2020-05-10', '2020-05-11']

    NUM_PARALLEL_INPUT_READ = 3

    SAVE_MODEL_INTERVAL_DAYS = 3

    DEEP_HIDDEN_UNITS = [512, 256, 256]

    WIDE_SLOTS = [ "1","2","3","4"]
    DEEP_SLOTS = [ "1","2","3","4"]

    PREDICT_DT = None
    PREDICT_DUMP_PATH = "./predict"


C = Config

def columns_builder():
    """Builds a set of wide and deep feature columns."""

    columns = {}
    for slot in set(C.WIDE_SLOTS + C.DEEP_SLOTS):
        columns[slot] = tn.feature_column.category_column(key=slot)

    wide_columns = []
    for slot in C.WIDE_SLOTS:
        feature_column = tf.feature_column.embedding_column(columns[slot], dimension=1)

        wide_columns.append(feature_column)

    deep_columns = []
    for slot in C.DEEP_SLOTS:
        feature_column = tf.feature_column.embedding_column(columns[slot], dimension=8)
        deep_columns.append(feature_column)

    return wide_columns, deep_columns


def parse_line_batch(example_proto):
    fea_desc = {
        "label": tf.io.FixedLenFeature([], tf.int64)
    }

    for slot in set(C.WIDE_SLOTS + C.DEEP_SLOTS):
        fea_desc[slot]  = tf.io.VarLenFeature(tf.int64)

    feature_dict = tf.io.parse_example(example_proto, fea_desc)

    # [batch_size, label]
    label = feature_dict.pop('label')

    return feature_dict, label


def read_dataset(data_path, days, match_pattern, num_parallel_calls = 12):
    ds_data_files = tn.data.list_files(data_path, days=days, match_pattern=match_pattern)
    dataset = ds_data_files.shard(num_shards=tn.core.shard_num(), index=tn.core.self_shard_id())
    dataset = dataset.interleave(lambda f: tf.data.TFRecordDataset(f, buffer_size=1024 * 100),
                                       cycle_length=4, block_length=8,
                                       num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(C.BATCH_SIZE)
    dataset = dataset.map(map_func=parse_line_batch, num_parallel_calls=num_parallel_calls)
    dataset = tn.data.BalanceDataset(dataset)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

## 创建切分版的模型

### embedding model

我们将获取embedding的部分切分到单独的model里去，在线inference的时候不需要加载这部分。在线逻辑实现中只要将TensorNet保存的sparse embedding数据导出成字典，然后根据每一条请求查出相应的embedding数据只会塞到我们下面提到的另外一个模型里去即可。

In [8]:
def create_emb_model(wide_columns, deep_columns):
    wide_embs, deep_embs = [], []

    inputs = {}
    for slot in set(C.WIDE_SLOTS + C.DEEP_SLOTS):
        inputs[slot] = tf.keras.layers.Input(name=slot, shape=(None,), dtype="int64", sparse=True)

    sparse_opt = tn.core.AdaGrad(learning_rate=0.01, initial_g2sum=0.1, initial_scale=0.1)

    if wide_columns:
        wide_embs = tn.layers.EmbeddingFeatures(wide_columns, sparse_opt, name='wide_inputs')(inputs)

    if deep_columns:
        deep_embs = tn.layers.EmbeddingFeatures(deep_columns, sparse_opt, name='deep_inputs')(inputs)

    # must put wide embs at front of outputs list
    emb_model = tf.keras.Model(inputs=inputs, outputs=[wide_embs, deep_embs], name="emb_model")

    return emb_model

### online inference model

inference model的输入是上个模型的输出，这个模型是我们在线预估真正用到的模型。

In [9]:
def create_sub_model(wide_emb_input_shapes, deep_emb_input_shapes):
    wide, deep = None, None

    wide_inputs = [tf.keras.layers.Input(name="wide_emb_{}".format(i), dtype="float32", shape=shape[1:])
                    for i, shape in enumerate(wide_emb_input_shapes)]

    deep_inputs = [tf.keras.layers.Input(name="deep_emb_{}".format(i), dtype="float32", shape=shape[1:])
                    for i, shape in enumerate(deep_emb_input_shapes)]

    if wide_inputs:
        wide = tf.keras.layers.Concatenate(name='wide_concact', axis=-1)(wide_inputs)

    if deep_inputs:
        deep = tf.keras.layers.Concatenate(name='deep_concact', axis=-1)(deep_inputs)

        for i, unit in enumerate(C.DEEP_HIDDEN_UNITS):
            deep = tf.keras.layers.Dense(unit, activation='relu', name='dnn_{}'.format(i))(deep)

    if wide_inputs and not deep_inputs:
        output = tf.keras.layers.Dense(1, activation='sigmoid', name='pred')(wide)
    elif deep_inputs and not wide_inputs:
        output = tf.keras.layers.Dense(1, activation='sigmoid', name='pred')(deep)
    else:
        both = tf.keras.layers.concatenate([deep, wide], name='both')
        output = tf.keras.layers.Dense(1, activation='sigmoid', name='pred')(both)

    model = tn.model.Model(inputs=[wide_inputs, deep_inputs], outputs=output, name="sub_model")

    return model

下面是整体模型

In [10]:
def create_model(wide_columns, deep_columns):
    inputs = {}
    for slot in set(C.WIDE_SLOTS + C.DEEP_SLOTS):
        inputs[slot] = tf.keras.layers.Input(name=slot, shape=(None,), dtype="int64", sparse=True)

    emb_model = create_emb_model(wide_columns, deep_columns)

    assert len(emb_model.output) == 2, "expected emb_model output length is 2 but {}".format(emb_model.output)
    wide_emb_input_shapes = [emb.shape for emb in emb_model.output[0]]
    deep_emb_input_shapes = [emb.shape for emb in emb_model.output[1]]

    wide_embs, deep_embs = emb_model(inputs)
    sub_model = create_sub_model(wide_emb_input_shapes, deep_emb_input_shapes)
    output = sub_model([wide_embs, deep_embs])
    model = tn.model.Model(inputs=inputs, outputs=output, name="full_model")

    dense_opt = tn.core.Adam(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8)
    model.compile(optimizer=tn.optimizer.Optimizer(dense_opt),
                  loss='binary_crossentropy',
                  metrics=['acc', "mse", "mae", 'mape', tf.keras.metrics.AUC(),
                           tn.metric.CTR(), tn.metric.PCTR(), tn.metric.COPC()])

    return model, sub_model

测试数据与[quick start with wide deep](01-begin-with-wide-deep.ipynb)相同

In [11]:
TEST_DATA_PATH = "/tmp/wide-deep-test/"

注意下面代码中我们只保存了sub_model，其它的参数`tn.callbacks.PsWeightCheckpoint`会自动保存。

In [18]:
def main():
    strategy = tn.distribute.PsStrategy()

    with strategy.scope():
        wide_column, deep_column = columns_builder()
        model, sub_model = create_model(wide_column, deep_column)

        train_dataset = read_dataset(TEST_DATA_PATH, [''], C.FILE_MATCH_PATTERN)

        cp_cb = tn.callbacks.PsWeightCheckpoint(C.MODEL_DIR, need_save_model=True, dt="")
        model.fit(train_dataset, epochs=1, verbose=1, callbacks=[cp_cb])

        infer_batch_size = 100
        for tensor in sub_model.inputs:
            tensor.set_shape([infer_batch_size] + list(tensor.shape)[1:])

        sub_model.save('/tmp/wide-deep-test/model/tmp')
    return

main()


750/750 [==============================] - 6s 8ms/step - loss: 0.6935 - acc: 0.4960 - mse: 0.2502 - mae: 0.5001 - mape: 249124448.0000 - auc_4: 0.4951 - CTR: 0.5036 - PCTR: 0.5018 - COPC: 1.0037
INFO:tensorflow:Assets written to: /tmp/wide-deep-test/model/tmp/assets


In [16]:
! ls /tmp/wide-deep-test/model/tmp

assets	saved_model.pb	variables


## 在线使用

模型保存成TensorFlow标准的SavedModel之后我们便可以使用TensorFlow提供的inference方案在线使用了，用户可以选择tf-serving的方案，或者选择XLA的方案，我们的在线逻辑使用XLA方案。

## 总结

在上面的例子中我们拿TensorNet在线预估的场景说明的切分模型的办法，相对于其它类型的模型其实也是一个思路，当切分完成之后按照具体需求保存即可。